# Data Carpentry Reproducible Research - Data Exploration Lesson

## Learning objectives
* Students should be able to assess the structure and cleanliness of their dataset, including size and shape of data, number of variables of each type 
* Students should be able to describe their findings, translate results from code to text using Markdown comments in the Jupyter Notebook, and summarize their thought process in a narrative
* Students should be able to modify the raw data to prepare a clean data set -- including copying data, removing or replacing missing and incoherent data, dropping columns, removing duplicates in Pandas and Jupyter -- and explain and justify their decisions in markdown in their Jupyter notebook
* Students should be able to assess whether their data is “Tidy” and identify appropriate steps and write and  execute code to arrange it into a tidy format - including merging, reshaping, subsetting, grouping, sorting, making appropriate new columns  - and explain and justify their decisions in markdown in their Jupyter notebook
* Students should be able to identify several relevant summary measures, illustrate data using appropriate plots, and explain and justify their decisions in markdown in their Jupyter notebook
* Student should assess the summaries and plots and appraise the need for repeated or further analysis, and justify decisions in markdown


# Setting up the notebook

## About Libraries in Python

A library in Python contains a set of tools (called functions) that perform tasks on our data. Importing a library is like getting a piece of lab equipment out of a storage locker and setting it up on the bench for use in a project. Once a library is imported, it can be used or called to perform many tasks.

Python doesn’t load all of the libraries available to it by default. We have to add an import statement to our code in order to use library functions. To import a library, we use the syntax `import libraryName`. If we want to give the library a nickname to shorten the command, we can add `as nickNameHere`. An example of importing the pandas library using the common nickname pd is below.

## matplotlib and other plotting libraries

matplotlib is the most widely used Python library for plotting.  We can run it in the notebook using the magic command `%matplotlib inline`.  See [the docs](http://ipython.readthedocs.io/en/stable/interactive/plotting.html) for other options to pass to the magic command.

In this lesson, we will only use matplotlib and seaborn, another package that works in tandem with matplotlib to make nice graphics.  There are a whole range of graphics packages in Python, ranging from basic visualizations to fancy, interactive graphics like [bokeh](http://bokeh.pydata.org/en/latest/) and [plotly](https://plot.ly/python/).  

We encourage you to explore on your own!  Chances are, if you can imagine a plot you'd like to make, somebody else has written a package to do it.

## The Pandas Library

One of the best options for working with tabular data in Python is to use the Python Data Analysis Library (a.k.a. Pandas). The Pandas library provides data structures, produces high quality plots with matplotlib and integrates nicely with other libraries that use NumPy (which is another Python library) arrays.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named 'pandas'

Each time we call a function that’s in a library, we use the syntax `LibraryName.FunctionName`. Adding the library name with a `.` before the function name tells Python where to find the function. In the example above, we have imported Pandas as `pd`. This means we don’t have to type out pandas each time we call a Pandas function.

We will begin by locating and reading our data which are in a table format. We can use Pandas’ `read_table` function to pull the file directly into a DataFrame.

# Getting data into the notebook

## What’s a DataFrame?
A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, factors and more) in columns. It is similar to a spreadsheet or an SQL table or the data.frame in R. A DataFrame always has an index (0-based). An index refers to the position of an element in the data structure.

Note that we use `pd.read_table`, not just `read_table` or `pandas.read_table`, because we imported pandas as pd.

The columns in the data set are separated by a TAB. We need to tell the `read_table` function in Pandas that that is the case with `sep = ‘\t’`.

The first thing to do when loading data into the notebook is to actually look at it.  How many rows and columns are there?  What types of variables are in it and what values can they take?

There are usually too many rows to print to the screen.  By default, when you type the name of the dataframe and run a cell, Pandas knows not to print the whole thing.  Instead, you will see the first and last few rows with dots in between.  A neater way to view a preview of the dataset is the `head()` method.  Calling `dataset.head()` will displace the first 5 rows of the data.  You can specify how many rows you want to see as an argument, like `dataset.head(10)`.  The `tail()` method does the same with the last rows of the dataframe.

In [2]:
url = "https://raw.githubusercontent.com/Reproducible-Science-Curriculum/data-exploration-RR-Jupyter/master/gapminderDataFiveYear_superDirty.txt"
gapminder = pd.read_table(url, sep = "\t")
gapminder.head()

NameError: name 'pd' is not defined

# Assess the structure and cleanliness


## How many rows and columns are in the data?
We often want to know how many rows and columns are in the data  --  we want to know what is called the "shape" attribute of the data frame. Pandas has a convenient way for getting that information by using the `DataFrame.shape` (using DataFrame as a generic name for a, well, data frame). This returns a tuple (values separated by commas) representing the dimensions of the DataFrame (rows, columns).<p>
To get the shape of the gapminder data frame:

In [ ]:
gapminder.shape

The `info()` method gives a few useful pieces of information, including the shape of the dataframe, the variable type of each column, and the amount of memory stored.

The output from info() shows that the fields ‘year’ and ‘pop’ (population) are represented as ‘float’ (that is: numbers with a decimal point). This is not appropriate: year and population should be integers or whole numbers. We can change the data-type with the function astype().

In [ ]:
gapminder.info()

In [ ]:
#gapminder['year'] = gapminder['year'].astype(int)
#gapminder['pop'] = gapminder['pop'].astype(int)
#gapminder.info()

The `describe()` method will take the numeric columns and give a summary of their values. This is useful for getting a sense of the ranges of values and seeing if there are any unusual or suspicious numbers.


In [ ]:
gapminder.describe()

The DataFrame function describe() just blindly looks at all numeric variables. We wouldn't actually want to take the mean year. Additionally, we obtain ‘NaN’ values for our quartiles. This suggests we might have missing data which we can (and will) deal with shortly when we begin to clean our data.

For now, let's pull out only the columns that are truly continuous numbers (i.e. ignore the description for ‘year’). This is a preview of selection columns of the data; we'll talk more about how to do it later in the lesson.

In [ ]:
gapminder[['pop', 'life Exp', 'gdpPercap']].describe()

The command value_counts() gives you a first global idea of your categorical data such as strings. In this case that is the column `region`.
The table reveals some problems. The data set covers 12 years, so each ‘region’ should appear 12 times.

In [ ]:
print(len(gapminder['region'].unique())) # How many unique regions are in the data?
gapminder['region'].value_counts() # How many times does each unique region occur?

We also see inconsistencies (string variables are very susceptible to those), for instance:

Asia_china	vs. Asia_China

Another type of problem is the various names of Congo. In order to analyze this dataset appropriately we need to take care of these issues. We also see that there are a few regions that occur more than 12 times.  We will address them in the next section on data cleaning.

# Data cleaning


## Handling Missing Data

Missing data is an important issue to handle. Ignoring it doesn't make it go away. There are different ways of dealing with missing data which include:

* analyzing only the available data (i.e. ignore the missing data)
* input the missing data with replacement values and treating these as though they were observed
* input the missing data and account for the fact that these were inputed with uncertainty (ex: create a new boolean variable so you know that these values were not actually observed)
* use statistical models to allow for missing data--make assumptions about their relationships with the available data as necessary

For our purposes with the dirty gapminder data set, we know our missing data is excess (and unnecessary) and we are going to choose to analyze only the available data. To do this, we will simply remove rows with missing values.

This is incredibly easy to do because pandas allows you to either remove all instances will null data or replace them with a particular value.

`df = df.dropna()` drops rows with any column having NA/null data.  `df = df.fillna(value)` replaces all NA/null data with the argument `value`.

In [ ]:
gapminder = gapminder.dropna()
gapminder.head()

## Handling (Unwanted) Repetitive Data
You can identify which observations are duplicates.
The call `df.duplicated()` will return boolean values for each row in the DataFrame telling you whether or not a row is repetitive.

In cases where you don’t want repetitive values (we wouldn’t--we only want each country to be represented once for every relevant year), you can easily drop such duplicate rows with the call `df.drop_duplicates()`.

In [ ]:
gapminder.duplicated().head() #shows we have a repetition in the first 5 rows

In [ ]:
gapminder = gapminder.drop_duplicates()
gapminder.head()

### Reindexing with reset_index()
Now we have 1704 rows, but our indexes are off. We can reset our indices easily with the call reset_index(drop=True). Remember, Python is 0-indexed so our indices will be valued 0-1703.

The concept of reindexing is important. When we removed some of the messier, unwanted data, we had "gaps" in our index values. By correcting this, we can improve our search functionality and our ability to perform iterative functions on our cleaned data set.

In [ ]:
gapminder = gapminder.reset_index(drop=True)
gapminder.head()


## Handling Inconsistent Data

The `region` column is a bit too messy for what we'd like to do.
The `value_counts()` operation above revealed some issues.

### String manipulations

Very common problems with string variables are  lingering white space and upper case vs. lower case.
The following three commands remove all such trailing spaces (left and right) and put everything in lowercase. If you prefer, the three commands can be written in one single line (concept: chaining). 

In [ ]:
gapminder['region'] = gapminder['region'].str.lstrip() # Strip white space on left
gapminder['region'] = gapminder['region'].str.rstrip() # Strip white space on right
gapminder['region'] = gapminder['region'].str.lower() # Convert to lowercase
gapminder['region'].value_counts() # How many times does each unique region occur?

# We could have done this in one line!
# gapminder['region'] = gapminder['region'].str.lstrip().str.rstrip().lower()

### regex + replace()

A regular expression, aka regex, is a sequence of characters that define a search pattern. In a regular expression, the symbol “*” matches the preceding character 0 or more times, whereas “+” matches the preceding character 1 or more times. “.” matches any single character. Writing “x|y” means to match either ‘x’ or ‘y’.

For more regex shortcuts (cheatsheet): https://www.shortcutfoo.com/app/dojos/regex/cheatsheet

Pandas allows you to use regex in its replace() function -- when a regex term is found in an element, the element is then replaced with the specified replacement term. In order for it to appropriately correct elements, both regex and inplace variables need to be set to True (as their defaults are false). This ensures that the initial input string is read as a regular expression and that the elements will be modified in place.

For more documentation on the replace method: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html

Here's an incorrect regex example: we create a temporary DataFrame in which a regex pulls all values that contain the term “congo”. Unfortunately, this creates 24 instances of the Democratic Republic of the Congo -- this is an error in our cleaning! We can revert back to the non-temporary DataFrame and correctly modify our regex to isolate only the Democratic Republic instances (as opposed to including the Republic as well).

In [ ]:
# This gives a problem -- 24 values of the congo!
temp = gapminder['region'].replace(".*congo.*", "africa_dem rep congo", regex=True)
temp.value_counts()

In [ ]:
# What happened? This shows all the rows that have congo in the name.
gapminder[gapminder["region"].str.contains('congo')]

### Using regex to fix the Congo...

As noted above, regular expressions (often simply "regex") provide a powerful 
tool for fixing errors that arise in strings. In order to correctly label the 
two different countries that include the word "congo", we need to design and
use (via `pd.df.replace()`) a regex that correctly differentiates between the
two countries.

Recall that the "." is the wildcard (matching any single character); combining 
this with "*" allows us to match any number of single characters an unspecified 
number of times. By combining these characters with substrings corresponding to
variations in the naming of the Democratic Republic of the Congo, we can
correctly normalize the name.

If you feel that the use of regex is not particularly straightforward, you are
correct -- appropriately using these tools takes a great deal of time to master.
When designing regex for these sorts of tasks, you might find the following
prototyper helpful: https://regex101.com/  

In [ ]:
gapminder['region'].replace(".*congo, dem.*", "africa_dem rep congo", regex=True, inplace=True)
gapminder['region'].replace(".*_democratic republic of the congo", "africa_dem rep congo", regex=True, inplace=True)
gapminder['region'].value_counts() # Now it's fixed.

### Exercise (regex):

Now that we've taken a close look at how to properly design and use regex to
clean string entries in our data, let's try to normalize the naming of a few
other countries. Using the pandas code we constructed above as a template,
construct similar code (using `pd.df.replace()`) to set the naming of the Ivory
Coast and Canada to "africa_cote d'ivoire" and "americas_canada", respectively.

In [ ]:
gapminder['region'].replace(".*ivore.*", "africa_cote d'ivoire", regex=True, inplace=True)
gapminder['region'].replace("^_canada", "americas_canada", regex=True, inplace=True)
gapminder['region'].value_counts()

## Tidy data

Having what is called a _Tidy_ data set can make cleaning your data much easier. Two of the important aspects of Tidy data are:
* every variable has its own column
* every observation has its own row

(there are other aspects of Tidy data, here is a good blog post about Tidy data in Python: http://www.jeannicholashould.com/tidy-data-in-python.html)

Currently the dataset has a single column for continent and country (the ‘region’ column). We can split that column into two, by using the underscore that separates continent from country.
We can create a new column in the DataFrame by naming it before the = sign:
`gapminder['country'] = `

The following commands use the function split() to split the string at the underscore (the first argument), which results in a list of two elements: before and after the \_. The second argument tells split() that the split should take place only at the first occurrence of the underscore.

In [ ]:
gapminder['country']=gapminder['region'].str.split('_', 1).str[1]
gapminder['continent']=gapminder['region'].str.split('_', 1).str[0]
gapminder.head()

### Removing and renaming columns

We have now added the columns `country` and `continent`, but we still have the old `region` column as well. In order to remove that column we use the `drop()` command. The first argument of the drop() command is the name of the element to be dropped. The second argument is the *axis* number: 0 for row, 1 for column.

In [ ]:
gapminder = gapminder.drop('region', 1)
gapminder.head()

Finally, it is a good idea to look critically at your column names. Use lowercase for all column names to avoid confusing `gdppercap` with `gdpPercap` or `GDPpercap`. Avoid spaces in column names to simplify manipulating your data - look out for lingering white space at the beginning or end of your column names. The following code turns all column names to lowercase. 


In [ ]:
gapminder.columns = gapminder.columns.str.lower()
gapminder.head()

We also want to remove the space from the `life exp` column name. We can do that with Pandas `rename` method. It takes a dictionary as its argument, with the old column names as keys and new column names as values.

If you're unfamiliar with dictionaries, they are a very useful data structure in Python.  You can read more about them [here](https://docs.python.org/3/tutorial/datastructures.html#dictionaries).

In [ ]:
gapminder = gapminder.rename(columns={'life exp' : 'lifeexp'})
gapminder.head()

## Merging data

Often we have more than one data frame that contains parts of our data set and we want to put them together. This is known as merging the data.

Our advisor now wants us to add a new country called The People's Republic of Berkeley to the gapminder data set that we have cleaned up. Our goal is to get this new data into the same data frame in the same format as the gapminder data and, in this case, we want to concatenate (add) it onto the end of the gapminder data.

Concatentating is a simple form of merging, there are many useful (and more complicated) ways to merge data.  If you are interested in more information, the [Pandas Documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html) is useful.

In [ ]:
PRB = pd.read_table('https://raw.githubusercontent.com/Reproducible-Science-Curriculum/data-exploration-RR-Jupyter/master/PRB_data.txt', sep = "\t")
PRB.head()

In [ ]:
## make this into an exercise
## bring in PRB data (no major problems) and make it conform to the gapminder at this point
# our version...
# clean the data to look like the current gapminder

PRB['country']=PRB['region'].str.split('_', 1).str[1].str.lower()
PRB['continent']=PRB['region'].str.split('_', 1).str[0].str.lower()
PRB = PRB.drop('region', 1)
PRB.columns = PRB.columns.str.lower()
PRB = PRB.rename(columns={'life exp' : 'lifeexp'})
PRB.head()

In [ ]:
# double check that the gapminder is the same
gapminder.head()

In [ ]:
# combine the data sets with concat
gapminder_comb = pd.concat([gapminder, PRB])
gapminder_comb.tail(15)

Now that the data frames have been concatenated, notice that the index is funky. It repeats the numbers 0 - 11 in the `peoples republic of berkeley data`. <p>
as an **exercise** fix the index.


In [ ]:
# our code for fixing index
gapminder_comb = gapminder_comb.reset_index(drop=True)
gapminder_comb.head()

## Referencing objects vs copying objects
Suppose you take a subset of your data and store it in a new variable, like `gapminder_early = gapminder[gapminder['year'] < 1970]`.  Doing this does not actually create a new object. Instead, you have just given a name to that subset of the original data: gapminder_early points to those rows of gapminder.  Any changes you make to the new dataframe gapminder_early will appear in the corresponding rows of gapminder too.  

Often, you want to leave the original data untouched.  To avoid this problem, you can make a copy of your data before operating on a subset of the data.  This will ensure that one version of the original data is preserved.  For instance:

<font color='red'>**TO DO:** this example doesn't actually illustrate the problem :(</font>

In [ ]:
gapminder_copy = gapminder.copy()
gapminder_early = gapminder_copy[gapminder_copy['year'] < 1970]
gapminder_early['pop'] = 0
#print(gapminder_copy[gapminder_copy['year'] < 1970].head())
#print(gapminder[gapminder['year'] < 1970].head())

## Subsetting and sorting

There are many ways in which you can manipulate a Pandas DataFrame - here we will discuss only only two: subsetting and sorting.

We can subset (or slice) by giving the numbers of the rows you want to see between square brackets:

In [ ]:
gapminder[0:15]

In [ ]:
gapminder[:15]

In [ ]:
gapminder[-10:]

### Exercise

What does the negative number (in the third cell) mean? What happens when you leave the space before or after the colon empty?

Subsetting can also be done by selecting for a particular value in a column; for instance select the rows that have ‘africa’ in the column ‘continent. Note the double equal sign: single equal signs are used in Python to assign something to a variable. The double equal sign is a comparison: the variable to the left has to be exactly equal to the string to the right.

**to do: are there other ways of subsetting that we want to talk about? .loc/.iloc with data frames**

In [ ]:
gapminder_africa = gapminder[gapminder['continent']=='africa']
gapminder_africa.head()

Sorting may help to further organize and inspect your data. The command sort_values() takes a number of arguments; the most important ones are `by` and `ascending.` The following command will sort your DataFrame by year, beginning with the most recent.

In [ ]:
gapminder.sort_values(by='year', ascending = False)

### Exercise

Organize your data set by country, from ‘Afganistan’ to ‘Zimbabwe’.

## Summarize and plot

Summaries (but can’t *say* statistics…)
* Sort data
* Can make note about using numpy functions, dif between dataframe and array
Good Plots for the data/variable type



Plots 
* of subsets, 
* single variables
* pairs of variables
* Matplotlib syntax (w/ seaborn for defaults (prettier, package also good for more analysis later...))

Exploring is often iterative - summarize, plot, summarize, plot, etc. - sometimes it branches…


# Summarizing data

Remember that the `info()` method gives a few useful pieces of information, including the shape of the dataframe, the variable type of each column, and the amount of memory stored. We can see many of our changes (continent and country columns instead of region, higher number of rows, etc.) reflected in the output of the `info()` method.

In [ ]:
gapminder_comb.info()

We also saw above that the `describe()` method will take the numeric columns and give a summary of their values. We have to remember that we changed the changed column names, and this time it shouldn't have NAs.

In [ ]:
gapminder_comb[['pop', 'lifeexp', 'gdppercap']].describe()

### More summaries

What if we just want a single value, like the mean of the population? We can call mean on a single column this way:


In [ ]:
gapminder_comb['pop'].mean()

What if we want to know the mean population by _continent_? Then we need to use the Pandas `groupby()` method and tell it which column we want to group by.


In [ ]:
gapminder_comb[['continent', 'pop']].groupby(by='continent').mean()

What if we want to know the mean population by continent?


In [ ]:
gapminder_comb[['continent', 'pop']].groupby(by='continent').median()

Or the number of entries (rows) per continent?


In [ ]:
gapminder_comb[['continent', 'country']].groupby(by='continent').count()

Sometimes we don't want a whole data frame. Here is another way to do this that produces a series as opposed to a data frame that tells us number of entries (rows). 


In [ ]:
gapminder_comb[['continent', 'country']].groupby(by='continent').size()

We can also look at the mean GDP per capita of each country: 


In [ ]:
gapminder_comb[['country', 'gdppercap']].groupby(by='country').mean().head(12)

What if we wanted a new data frame that just contained these summaries? This could be a table in a report, for example.

In [ ]:
continent_mean_pop = gapminder_comb[['continent', 'pop']].groupby(by='continent').mean()
continent_mean_pop = continent_mean_pop.rename(columns = {'pop':'meanpop'})
continent_row_ct = gapminder_comb[['continent', 'country']].groupby(by='continent').count()
continent_row_ct = continent_row_ct.rename(columns = {'country':'nrows'})
continent_median_pop = gapminder_comb[['continent', 'pop']].groupby(by='continent').median()
continent_median_pop = continent_median_pop.rename(columns = {'pop':'medianpop'})
gapminder_summs = pd.concat([continent_row_ct,continent_mean_pop,continent_median_pop], axis=1)
gapminder_summs = gapminder_summs.rename(columns = {'y':'year'})
gapminder_summs

## Visualization with `matplotlib`

Recall that [matplotlib](http://matplotlib.org) is Python's main visualization 
library. It provides a range of tools for constructing plots, and numerous 
high-level plotting libraries (e.g., [seaborn](http://seaborn.pydata.org)) are 
built with matplotlib in mind. When we were in the early stages of setting up 
our analysis, we loaded these libraries like so:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

_Consider the above three commands to be essential practice for plotting (as
essential as `import pandas as pd` is for data munging)._

Now, let's turn to data visualization. In order to get a feel for the properties
of the data set we are working with, data visualization is key. While, we will
focus only on the essentials of how to properly construct plots in univariate
and bivariate settings here, it's worth noting that both matplotlib and seaborn
support a diversity of plots: [matplotlib 
gallery](http://matplotlib.org/gallery.html), [seaborn
gallery](http://seaborn.pydata.org/examples/). 

---

### Single variables

* __Histograms__ - provide a quick way of visualizing the distribution of numerical
  data, or the frequencies of observations for categorical variables.


In [ ]:
# example plot goes here

* __Boxplots__ - provide a way of comparing the summary measures (e.g., max, min,
  quartiles) across variables in a data set.

```Python
# example plot goes here
```

---

### Pairs of variables

* __Scatterplots__ - visualization of relationships across two variables...

```Python
# example plot goes here
```

* ...

```Python
# example plot goes here
```

---

### Why use `seaborn`?

As noted above, seaborn is a high-level plotting library for statistical data 
visualization. In addition to simplifying plotting, it also provides facilities 
for customizing matplotlib plots (accessible via `sns.set()`).

## Interpret plots and summaries

### Exploration is an iterative process

In this section, we've taken the raw data and worked through steps to prepare it for analysis, but we have not yet done any "data analysis".  This part of the data workflow can be thought of as "exploratory data analysis", or EDA.  Many of the steps we've shown are aimed at uncovering interesting or problematic things in the dataset that are not immediately obvious.  We want to stress that when you're doing EDA, it will not necessarily be a linear workflow like what we have shown.  When you plot or summarize your data, you may uncover new issues: we saw this when we made a mistake fixing the naming conventions for the Democratic Republic of Congo.  You might discover outliers, unusually large values, or points that don't make sense in your plots.  Clearly, the work here isn't done: you'll have to investigate these points, decide how to fix any potential problems, document the reasoning for your actions, and check that your fix actually worked.

On the other hand, plots and summaries might reveal interesting questions about your data.  You may return to the cleaning and prepping steps in order to dig deeper into these questions.  You should continuously refine your plots to give the clearest picture of your hypotheses.

### Interesting findings

This should be particular to the dataset at hand.  Need to build upon results from the previous section.
